# Assignment: Create a Tool-Enabled Research Assistant using CrewAI



---

### Objective:
This assignment challenges you to build a **multi-agent research assistant** using **CrewAI** that can leverage **external tools** to gather and synthesize information. You will define multiple AI agents with distinct roles (e.g., Researcher, Analyst, Report Writer), equip them with tools (like internet search), and orchestrate their collaboration to produce a comprehensive research report on a given topic. This will demonstrate how to enable agents to interact with the real world and combine their findings to achieve a complex goal.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API (e.g., Google's Gemini, OpenAI's GPT-4). For this assignment, we'll primarily use **Google's Gemini Pro model**.
2.  **External Tool Access**: You **must** use an internet search tool. **SerperDevTool** from `crewai_tools` is recommended due to its ease of integration. You will need a Serper API key for this.
3.  **Environment Setup**: Install the necessary Python libraries: `pip install crewai crewai_tools google-generativeai`.
4.  **API Keys**: Securely handle your LLM API key and your Serper API key. It's best practice to load them from environment variables.
5.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
6.  **Research Topic**: You will define a specific, current, and moderately complex research topic (e.g., "The future of personalized medicine through AI" or "Impact of quantum computing on cybersecurity").
7.  **Analysis**: Critically evaluate the research process, the agents' collaboration, and the quality of the final output.

---

## Part 1: Setup and LLM/Tool Configuration
Begin by installing necessary libraries and configuring your LLM and external tools for CrewAI.

### Task 1.1: Install Libraries and Configure LLM/Tools
Install `crewai`, `crewai_tools`, and `google-generativeai`. Set up your Google API key and **Serper API key** (essential for search) and initialize the LLM and search tool.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install crewai crewai_tools google-generativeai --quiet

import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool # Tool for internet search
from langchain_google_genai import ChatGoogleGenerativeAI

# --- YOUR API KEYS HERE ---
# It's highly recommended to load your API keys from environment variables for security.
# On Linux/macOS: export GOOGLE_API_KEY='your_key' ; export SERPER_API_KEY='your_key'
# On Windows (cmd): set GOOGLE_API_KEY=your_key & set SERPER_API_KEY=your_key

os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE" # Replace with your actual Google API key
os.environ["SERPER_API_KEY"] = "YOUR_SERPER_API_KEY_HERE" # Replace with your actual Serper API key (Get one from serper.dev)

# Initialize the LLM (Gemini Pro)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3) # Lower temperature for factual accuracy

# Initialize the search tool
search_tool = SerperDevTool()

print("CrewAI environment, LLM, and Serper Search Tool configured!")

---

## Part 2: Define Agents (Roles and Tools)
Create the specialized AI agents that will form your research team. Each agent will have a distinct role and some will be equipped with tools.

### Task 2.1: The Research Agent
Define an `Agent` whose role is to conduct thorough internet research.

* **Role**: `Senior Research Analyst`
* **Goal**: To conduct comprehensive and accurate research on complex topics using search tools.
* **Backstory**: A meticulous researcher with a vast understanding of information retrieval and verification. Excels at finding relevant and credible sources.
* **Tools**: Assign `search_tool`.
* **LLM**: Use the initialized `llm`.
* **Verbose**: Set to `True`.
* **Allow Delegation**: Set to `False`.

In [ ]:
researcher = Agent(
    role='Senior Research Analyst',
    goal='Conduct comprehensive and accurate research on complex topics using search tools.',
    backstory=(
        "A meticulous researcher with a vast understanding of information retrieval "
        "and verification. Excels at finding relevant and credible sources to "
        "support in-depth analysis."
    ),
    tools=[search_tool], # Assign the search tool here
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("Researcher Agent created!")

### Task 2.2: The Data Analyst Agent
Define an `Agent` whose role is to analyze research findings and extract key data points and trends.

* **Role**: `Data & Trend Analyst`
* **Goal**: To analyze research findings, identify key data points, trends, and potential implications.
* **Backstory**: An expert in data interpretation and pattern recognition. Transforms raw research into structured insights and actionable information.
* **LLM**: Use the initialized `llm`.
* **Verbose**: Set to `True`.
* **Allow Delegation**: Set to `False`.

In [ ]:
analyst = Agent(
    role='Data & Trend Analyst',
    goal='Analyze research findings, identify key data points, trends, and potential implications.',
    backstory=(
        "An expert in data interpretation and pattern recognition. "
        "Transforms raw research findings into structured insights and actionable information, "
        "highlighting what's important for the report."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("Analyst Agent created!")

### Task 2.3: The Report Writer Agent
Define an `Agent` whose role is to synthesize all findings into a final, polished research report.

* **Role**: `Technical Report Writer`
* **Goal**: To write a clear, concise, and well-structured research report based on the analyzed information.
* **Backstory**: A skilled technical writer with a knack for transforming complex analyses into readable and engaging reports. Ensures accuracy, coherence, and professional presentation.
* **LLM**: Use the initialized `llm`.
* **Verbose**: Set to `True`.
* **Allow Delegation**: Set to `False`.

In [ ]:
writer = Agent(
    role='Technical Report Writer',
    goal='Write a clear, concise, and well-structured research report based on the analyzed information.',
    backstory=(
        "A skilled technical writer with a knack for transforming complex analyses "
        "into readable and engaging reports. Ensures accuracy, coherence, and professional presentation."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("Report Writer Agent created!")

---

## Part 3: Define Tasks and Form the Crew
Define the specific tasks for each agent, ensuring proper context passing, and then assemble them into a `Crew`.

### Task 3.1: Define Research Task
Define the task for the `Researcher` agent. This task will involve using the `search_tool`.

* **Description**: "Conduct in-depth research on the given topic: `{topic}`. Gather factual information, statistics, historical context, and recent developments. Focus on identifying key challenges, opportunities, and future outlooks. Compile your findings into a comprehensive research brief."
* **Agent**: `researcher`
* **Output File (Optional)**: Save the raw research findings for inspection (e.g., `research_brief.md`).

In [ ]:
research_topic = "The future of personalized medicine through AI" # Your chosen research topic

research_task = Task(
    description=f"Conduct in-depth research on the given topic: '{research_topic}'. "
                "Gather factual information, statistics, historical context, and recent developments. "
                "Focus on identifying key challenges, opportunities, and future outlooks. "
                "Compile your findings into a comprehensive research brief that can be used for analysis.",
    agent=researcher,
    expected_output="A detailed research brief containing all relevant information, facts, and figures for the given topic.",
    output_file='research_brief.md' # Optional: save output to file
)

print("Research Task created!")

### Task 3.2: Define Analysis Task
Define the task for the `Analyst` agent. This task will take the output of the `research_task` as its context.

* **Description**: "Analyze the provided research brief: `{{ {{ research_task.output }} }}`. Extract the most critical data points, identify overarching trends, and infer potential implications for the future. Summarize these key insights concisely."
* **Agent**: `analyst`
* **Context**: Pass the output of `research_task`.
* **Output File (Optional)**: Save the analysis findings (e.g., `analysis_report.md`).

In [ ]:
analysis_task = Task(
    description=f"Analyze the provided research brief on '{research_topic}': '{{ {{ research_task.output }} }}'. "
                "Extract the most critical data points, identify overarching trends, and infer potential implications for the future. "
                "Summarize these key insights concisely, ready for inclusion in a final report.",
    agent=analyst,
    context=[research_task], # This links the output of research_task as input to analysis_task
    expected_output="A concise summary of key data points, trends, and implications from the research.",
    output_file='analysis_report.md' # Optional: save output to file
)

print("Analysis Task created!")

### Task 3.3: Define Report Writing Task
Define the task for the `Report Writer` agent. This task will take the output of the `analysis_task` as its context.

* **Description**: "Write a comprehensive and professional research report on `{topic}` based on the following analysis findings: `{{ {{ analysis_task.output }} }}`. The report should include an introduction, sections covering key findings/trends, implications, and a conclusion. Ensure the report is well-structured, clear, and around 800-1200 words. Maintain a formal and informative tone."
* **Agent**: `writer`
* **Context**: Pass the output of `analysis_task`.
* **Output File**: Save the final report (e.g., `final_research_report.md`).

In [ ]:
report_task = Task(
    description=f"Write a comprehensive and professional research report on '{research_topic}' based on the following analysis findings: '{{ {{ analysis_task.output }} }}'. "
                "The report should include an introduction, sections covering key findings/trends, implications, and a conclusion. "
                "Ensure the report is well-structured, clear, and approximately 800-1200 words. "
                "Maintain a formal and informative tone. "
                "Do not include any conversational filler, just the report content.",
    agent=writer,
    context=[analysis_task], # This links the output of analysis_task as input to report_task
    expected_output="A well-structured and comprehensive research report on the specified topic.",
    output_file='final_research_report.md' # Save the final report
)

print("Report Writing Task created!")

### Task 3.4: Form and Execute the Crew
Assemble your agents and tasks into a `Crew` and kick off the workflow.

* **Agents**: List `researcher`, `analyst`, and `writer`.
* **Tasks**: List `research_task`, `analysis_task`, and `report_task` in the correct sequential order.
* **Process**: Use `Process.sequential`.
* **Verbose**: Set to `True` to see detailed execution logs.

In [ ]:
research_crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, report_task],
    process=Process.sequential,
    verbose=True
)

print("\n--- Kicking off the Research Assistant Crew! ---")

# You can pass initial inputs here if your first task requires it
inputs_for_crew = {
    'topic': research_topic
}

result = research_crew.kickoff(inputs=inputs_for_crew)

print("\n--- Workflow Finished! ---")
print("Final Research Report Result (first 1000 chars):\n")
print(result[:1000] + "...")

print("\nCheck 'research_brief.md', 'analysis_report.md', and 'final_research_report.md' for full outputs.")

---

## Part 4: Analysis and Reflection
Examine the outputs generated by your CrewAI workflow and reflect on the effectiveness of tool-enabled multi-agent systems.

### Task 4.1: Review the Outputs
Inspect the `research_brief.md`, `analysis_report.md`, and `final_research_report.md` files (or the printed output) and answer the following questions.

* **Research Quality**: Did the `Researcher` agent effectively use the `SerperDevTool` to gather relevant information? Was the `research_brief.md` comprehensive and factual? Did it cover the key aspects of your chosen topic?
* **Analysis Quality**: Did the `Analyst` agent successfully synthesize the research brief into meaningful insights and trends? Was `analysis_report.md` concise and clear?
* **Report Quality**: Is the `final_research_report.md` well-structured, coherent, and informative? Does it accurately reflect the findings from the previous stages? Did it meet the length and tone requirements?
* **Collaboration and Tool Use**: Describe how the agents collaborated. Specifically, how did the `Researcher`'s use of the tool impact the `Analyst`'s work, and how did the `Analyst`'s output inform the `Report Writer`?
* **Challenges Encountered**: Did you observe any issues (e.g., outdated information, irrelevant search results, agents getting stuck, LLM hallucinations)? How might you address them in a production system?

### Task 4.2: Reflection on Tool-Enabled Agents
Consider the broader implications and benefits of equipping agents with external tools.

* **Power of Tools**: How do external tools enhance the capabilities of AI agents? What would be the limitations without them?
* **Agent Specialization**: How important was it to define distinct roles and responsibilities for each agent in this workflow? Could a single agent with all tools achieve the same quality of output?
* **Real-world Applications**: Beyond research, what other types of complex tasks could benefit from a tool-enabled multi-agent system (e.g., customer support, code generation, financial analysis)?
* **Future Enhancements**: If you were to improve this research assistant, what additional tools or agent capabilities would you add (e.g., a summarization tool, a fact-checking tool, a human feedback loop)?
* **Ethical Considerations**: Briefly discuss any ethical considerations that arise when AI agents are empowered with broad access to information via search tools (e.g., misinformation, bias in search results).

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* **Attach** the generated `research_brief.md`, `analysis_report.md`, and `final_research_report.md` files alongside your notebook if submitting to a platform that allows it, or copy their contents directly into markdown cells if not.
* Save your Jupyter Notebook as `[YourName]_CrewAI_Research_Assistant_Assignment.ipynb`.